In [1]:
'''Connecting to DB'''
import pyodbc
Server='qa-datagov01.nexjqa.local'
DB='RPO_MAJIK_DISCOVERY'
Ver='sql2012'
user='nexj'
password='nexj'

cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+Server+';DATABASE='+DB+';UID='+user+';PWD='+password)
# Create a cursor from the connection
cursor = cnxn.cursor()

In [2]:
#first we need to read all of the error codes
sqlStr='SELECT [id]  FROM [RPO_MAJIK_DISCOVERY].[dbo].[errorcode]'
cursor.execute(sqlStr)    
buffers = cursor.fetchall()
errorcodes=[]
for buffer in buffers: 
    errorcodes.append (buffer[0])
#print (errorcodes)  
# Read the equipment 
#
sqlStr='SELECT [id]  FROM [RPO_MAJIK_DISCOVERY].[dbo].[equipment]'  
cursor.execute(sqlStr)    
buffers = cursor.fetchall()
equipments=[]
eqStr="("

for buffer in buffers: 
    equipments.append (buffer[0]) 
    eqStr+='[E'+buffer[0].replace("-", "0")+']'# becuase thats how we name the columns in SQL 
    eqStr+=','


eqStr+="[errorID])"
#print (eqStr)


#Testing to write 
# sqlStr='INSERT INTO [RPO_MAJIK_DISCOVERY].[dbo].[mtTB]'+eqStr+' VALUES' +'(\'1\', \'1\',\'1\', \'1\',\'1\', \'1\',\'1\', \'1\',\'1\', \'1\',\'1\', \'1\',\'1\', \'1\',\'1\', \'1\',\'1\', \'1\',\'1\', \'1\',\'1\', \'1\',\'1\', \'1\',\'1\', \'1\',\'1\', \'1\',\'1\', \'1\',\'1\', \'1\',\'1\', \'1\',\'1\', \'1\')'+';'
# # print (sqlStr) 
# cursor.execute(sqlStr)   
# cnxn.commit()


([E3937001],[E3936001],[E3995001],[E3980006],[E3980005],[E3980004],[E3980003],[E3980002],[E3980001],[E3952001],[E3952002],[E3970007],[E3970008],[E3952010],[E3952003],[E3952004],[E3952005],[E3952006],[E3952007],[E3952008],[E3975002],[E3970002],[E3970003],[E3970004],[E3970006],[E3952009],[E3975001],[E3980008],[E3980007],[E3952013],[E3952014],[E3985001],[E3975003],[E3952011],[E3952012],[errorID])


In [34]:
'''Calculating the mean time to failure(MTTF) '''
from datetime import datetime

c=0

for error in errorcodes:

    c+=1
    if c> 20:
        break
     
    strWrite="(" 

    for equipment in equipments:
        
        sqlStr=' select * FROM [RPO_MAJIK_DISCOVERY].[dbo].[error]  WHERE LTRIM(RTRIM([equipmentId]))=LTRIM(RTRIM(\''+equipment+'\')) AND LTRIM(RTRIM([errorcodeId]))=LTRIM(RTRIM(\''+error+'\')) order by start_time'
#         print (sqlStr)
        cursor.execute(sqlStr)    
        '''Read all at once '''
        rows = cursor.fetchall()
        prev=0  # previouse value
        avg=0
        count=0
        for row in rows:
            if prev==0:
                prev =datetime.strptime(row[3][:16], '%Y-%m-%d %H:%M')
                
            startTime=datetime.strptime(row[3][:16], '%Y-%m-%d %H:%M')
            
            count+=1
            
            avg+=(startTime-prev).total_seconds()/3600        
            prev==datetime.strptime(row[4][:16], '%Y-%m-%d %H:%M')
#             print("-------------")
#             print(startTime)
#             print("prev")
#             print(prev)
#             print(avg)
#             print("-------------")
        if(count==0):
            count=1
        avg=avg/count
        strWrite+="\'"+str(avg)+"\'"+','
#         print("error="+error+"\n equipment="+equipment+"\n Avg="+str(avg)+"\n count"+str(count)+"\n\n")

           
            

    strWrite+="\'"+error+"\'"+")"
        
#     print (strWrite)
    sqlStr='INSERT INTO [RPO_MAJIK_DISCOVERY].[dbo].[mtTB]'+eqStr+' VALUES'+strWrite+';'
    print (sqlStr) 
    strWrite=""



#TODO write to SQL and update the table 
#CHANGE THE TABLE NAME AND CONDITIONS 









('0.0','0.0','0.0','1893.7333333333333','2236.5125','193.66666666666669','3646.1041666666665','1704.6791666666666','1712.611111111111','0.0','0.0','0.0','0.0','0.0','0.0','0.0','0.0','0.0','0.0','0.0','0.0','0.0','0.0','0.0','0.0','0.0','0.0','3789.977083333333','317.125','0.0','0.0','0.0','0.0','0.0','0.0','57489737814c9f11b33695ac')
INSERT INTO [RPO_MAJIK_DISCOVERY].[dbo].[mtTB]([E3937001],[E3936001],[E3995001],[E3980006],[E3980005],[E3980004],[E3980003],[E3980002],[E3980001],[E3952001],[E3952002],[E3970007],[E3970008],[E3952010],[E3952003],[E3952004],[E3952005],[E3952006],[E3952007],[E3952008],[E3975002],[E3970002],[E3970003],[E3970004],[E3970006],[E3952009],[E3975001],[E3980008],[E3980007],[E3952013],[E3952014],[E3985001],[E3975003],[E3952011],[E3952012],[errorID]) VALUES('0.0','0.0','0.0','1893.7333333333333','2236.5125','193.66666666666669','3646.1041666666665','1704.6791666666666','1712.611111111111','0.0','0.0','0.0','0.0','0.0','0.0','0.0','0.0','0.0','0.0','0.0','0.0','0.0','

KeyboardInterrupt: 

In [65]:
from datetime import datetime

fmt = '%Y-%m-%d %H:%M:%S'
d1 = datetime.strptime('2010-01-01 12:30:22', fmt)
d2 = datetime.strptime('2010-01-03 17:31:22', fmt)

#  print ((d2-d1).days*24)
print ((d2-d1).total_seconds()/3600)

2 days, 5:01:00
53.016666666666666


In [32]:
sqlStr='INSERT INTO [RPO_MAJIK_DISCOVERY].[dbo].[mtTB]'+eqStr+' VALUES' +'(\'1\', \'1\',\'1\', \'1\',\'1\', \'1\',\'1\', \'1\',\'1\', \'1\',\'1\', \'1\',\'1\', \'1\',\'1\', \'1\',\'1\', \'1\',\'1\', \'1\',\'1\', \'1\',\'1\', \'1\',\'1\', \'1\',\'1\', \'1\',\'1\', \'1\',\'1\', \'1\',\'1\', \'1\',\'1\', \'1\')'+';'
print (sqlStr) 

INSERT INTO [RPO_MAJIK_DISCOVERY].[dbo].[mtTB]([E3937001],[E3936001],[E3995001],[E3980006],[E3980005],[E3980004],[E3980003],[E3980002],[E3980001],[E3952001],[E3952002],[E3970007],[E3970008],[E3952010],[E3952003],[E3952004],[E3952005],[E3952006],[E3952007],[E3952008],[E3975002],[E3970002],[E3970003],[E3970004],[E3970006],[E3952009],[E3975001],[E3980008],[E3980007],[E3952013],[E3952014],[E3985001],[E3975003],[E3952011],[E3952012],[errorID]) VALUES('1', '1','1', '1','1', '1','1', '1','1', '1','1', '1','1', '1','1', '1','1', '1','1', '1','1', '1','1', '1','1', '1','1', '1','1', '1','1', '1','1', '1','1', '1');
